# Task04. 下游任务的端到端学习

# 1. 端到端学习的重要性

### 在Task04中我们使用PyPOTS对自定义数据集进行了两阶段建模, 即在上游先对数据集中的缺失值进行插补, 然后对处理好的数据集进行下游任务建模. 同时, 我们也强调了缺失值本身也是数据集的特性, 数据的缺失模式可能携带了额外的信息来表征数据采集对象的状态, 而这些信息在两阶段处理中可能会丢失, 因为插补值是模型根据观测到的数据分布推测出来的, 插补后下游算法无法知道原数据中的缺失模式, 也就无法充分利用这部分信息来学习. 端到端学习则是使用一个模型直接接受包含缺失值的数据然后在特定任务上进行学习...

# 2. 使用SAITS和TimesNet直接在自定义数据集上进行分类